In [2]:
import json
with open("all_v1.json", "r") as file:
    data = json.load(file)

In [4]:
#Test API

import requests
headers = {
    'X-RapidAPI-Host': 'ipeirotis-readability-metrics.p.rapidapi.com',
    'X-RapidAPI-Key': 'f9ubYNvChSmsh7ziUqOiramFPcBnp1UxuM2jsnCqrPp4oiO7Dw'
}
r = requests.post("https://ipeirotis-readability-metrics.p.rapidapi.com/getReadabilityMetrics", headers=headers, data={"text":"test"})
response = r.json()
response

{'FLESCH_KINCAID': -3.4,
 'SYLLABLES': 1.0,
 'COMPLEXWORDS': 0.0,
 'CHARACTERS': 4.0,
 'SENTENCES': 1.0,
 'FLESCH_READING': 121.22,
 'GUNNING_FOG': 0.4,
 'WORDS': 1.0,
 'COLEMAN_LIAU': -22.24,
 'ARI': -2.09,
 'SMOG_INDEX': 3.0,
 'SMOG': 3.129}

In [5]:
# Extractive summarization

import re
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
import string
import unidecode
import json
from bs4 import BeautifulSoup
import requests
LANGUAGE = 'english'
SENTENCES_COUNT = 15
clause = re.compile("(will)|(agree)|(must)|(responsib)|(waive)|(lawsuit)|(modify)|(intellec)", re.IGNORECASE)
host_reg = re.compile('(?:http.*://)?(?P<host>[^:/ ]+).?(?P<port>[0-9]*).*')
terms_page = re.compile(r"(terms *(((and|&)? *conditions)|((of)? ?(service|use))))", re.IGNORECASE)
HTML_OPEN = "<div id='mainPopup'>"
YOU_AGREE_HEADER = "<div id='youAgree' class='header'>What You Agree</div>"
THEY_AGREE_HEADER = "<div id='theyAgree' class='header'>What They Agree</div>"
OTHER_HEADER = "<div id='other' class='header'>Other Clauses</div>"

def prepare_for_regex(input_str, delimiter="."):
    pure_sentences = input_str.strip().split(delimiter)
    if pure_sentences[-1] == "":
        pure_sentences.pop()

    # Remove leading/trailing whitespace and end with period.
    for i, x in enumerate(pure_sentences):
        pure_sentences[i] = x.strip() + "."

    clean_sentences = []
    for x in pure_sentences:
        clean_sentences.append(' '.join(x.lower().translate(str.maketrans({key: None for key in string.punctuation})).split()))
    return clean_sentences, pure_sentences

In [6]:
link = "https://www.tech.gov.sg/privacy/"
r = requests.get(link)
html_soup = BeautifulSoup(r.text, "html.parser")
text = html_soup.text
        
text_data = unidecode.unidecode(text)
clean_list, pure_list = prepare_for_regex(text_data)

data_to_summarize = []
for clean, pure in zip(clean_list, pure_list):
    if re.findall(clause, clean):
        data_to_summarize.append(pure)
text_data = " ".join(data_to_summarize)
parser = PlaintextParser(text_data, Tokenizer(LANGUAGE))
stemmer = Stemmer(LANGUAGE)
summarizer = TextRankSummarizer(stemmer)

summary = summarizer(parser.document, SENTENCES_COUNT)

In [7]:
summary

(<Sentence: Most web browsers automatically accept cookies, but you can usually modify your browser setting to decline cookies if you prefer.>,
 <Sentence: (b) We will NOT share your Personal Data with non-Government entities, except where such entities have been authorised to carry out specific Government services.>,
 <Sentence: This will speed up the transaction and save you the trouble of repeating previous submissions.>,
 <Sentence: We are not responsible for the content and privacy practices of these other websites and encourage you to consult the privacy notices of those sites.>)

In [8]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
sentences = [str(sentence) for sentence in summary]

In [9]:
from IPython.core.display import HTML,display
all_sentences = []
for sent in sentences:
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    sent = re.sub(cleanr, '', sent)
    sent = nlp(sent)
    sentence = ""
    entities = []
    for token in sent:
        if sentence and token.text.strip() not in string.punctuation:
            sentence += " "
        if token.pos_ == "VERB":
            sentence += '''<mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone"> {}</mark>'''.format(token.text)
            entities.append(token.text)
        else:
            sentence += token.text
    sentence = sentence[:-1] + ". "
    for ent in sent.ents:
        if ent.text not in entities and ent.label_ == "ORG":
            sentence = sentence.replace(ent.text, '''<mark class="entity" style="background: #ccffcc; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone"> {} <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; text-transform: uppercase; vertical-align: middle; margin-left: 0.5rem">ORG</span></mark> '''.format(ent.text))
            entities.append(ent.text)
    all_sentences.append(sentence)
display(HTML("<br><br>".join(all_sentences)))

In [ ]:
#Scraping script

import requests
from bs4 import BeautifulSoup
r = requests.get("https://tosdr.org/index.html#")
soup = BeautifulSoup(r.text, "html.parser")
spans = soup.findAll("div", class_="span6")
data = []
classi = {
    "badge-warning": -1,
    "badge-neutral": 0,
    "badge-success": 1,
    "badge-important": -1
}
for span in spans:
    for item in span.findAll("li"):
        classification = item.span['class']
        classification.remove('badge')
        text = item.text[2:]
        data.append({
            "text": text,
            "rating": classi[classification[0]]
        })
with open("sentiment_data.json", "w") as file:
    json.dump(data, file)

In [13]:
# Test API endpoint
import requests
link = "https://hacknroll.nushackers.org/privacy"
r = requests.post("https://d12aodjr8sssf3.cloudfront.net/", json={"link":link})
r.json()

{'complexity_score': 0.11855670103092783,
 'readability_score': 85.158,
 'summary_sentences': [{'rating': -1,
   'text': 'WITHDRAWING YOUR CONSENTThe <mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">consent</mark> that you <mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">provide</mark> for the <mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">collection</mark>, use and <mark class="entity" style="background: #ffffb3; padding: 0.2em 0.2em; line-height: 1; border-radius: 0.35em; box-decoration-break: clone; -webkit-box-decoration-break: clone">disclosure</mark> of your <mark class="entity" style="background: #b3d9f